In [1]:
import pandas
import numpy



In [5]:
def load_data():
    path = './data.csv'
    df = pandas.read_csv(path)
    df['Anomaly'] = df.Anomaly.astype(str)
    df['Date'] = pandas.to_datetime(df.Date)
    df = df.sort_values('Date')
    return df

data = load_data()
data

,Date,Count,Anomaly
0,2019-09-29,11462,nan
1,2019-10-30,12782,nan
2,2019-11-28,686,anomaly
3,2020-02-28,13222,nan
4,2020-03-30,13305,nan
5,2020-04-29,13316,nan
6,2020-05-30,14016,anomaly
7,2020-06-29,13372,nan
8,2020-07-30,13487,nan
9,2020-08-30,13519,nan


In [71]:
def detect_out_of_order(df):
    out = pandas.Series([None]*len(df), index=df.index, name='OutOfOrder')
    
    for i in range(len(df)):
        #print(i, len(df))
        prev = df.iloc[i-1] if i > 0 else None
        next = df.iloc[i+1] if i < (len(df)-1) else None
        current = df.iloc[i]
        
        in_order = None
        if prev and next:
            if prev > current:
                #out.loc[i] = False
                pass
            elif next < current:
                out.loc[i+1] = False
                pass
            else:
                pass
                #out.loc[i] = True
            in_order = (prev <= current) and (next >= current)
            print(i, prev, prev > current, current, next, in_order)
        
    return out

data = load_data()
oo = detect_out_of_order(data.Count)
oo

data1 = pandas.merge(data, oo, right_index=True, left_index=True)
#data.loc[oo.index, 'Detect'] = oo 
data1

1 11462 False 12782 686 False
2 12782 True 686 13222 False
3 686 False 13222 13305 True
4 13222 False 13305 13316 True
5 13305 False 13316 14016 True
6 13316 False 14016 13372 False
7 14016 True 13372 13487 False
8 13372 False 13487 13519 True
9 13487 False 13519 13553 True
10 13519 False 13553 686 False
11 13553 True 686 13577 False
12 686 False 13577 13580 True
13 13577 False 13580 13594 True
14 13580 False 13594 13594 True
15 13594 False 13594 686 False
16 13594 True 686 686 False
17 686 False 686 13619 True
18 686 False 13619 13619 True


,Date,Count,Anomaly,OutOfOrder
0,2019-09-29,11462,nan,None
1,2019-10-30,12782,nan,None
2,2019-11-28,686,anomaly,False
3,2020-02-28,13222,nan,None
4,2020-03-30,13305,nan,None
5,2020-04-29,13316,nan,None
6,2020-05-30,14016,anomaly,None
7,2020-06-29,13372,nan,False
8,2020-07-30,13487,nan,None
9,2020-08-30,13519,nan,None


In [56]:
oom = (data.Count.diff(1) < 0).rename('OutOfOrder')
data2 = pandas.merge(data, oom, right_index=True, left_index=True)
#data.loc[oo.index, 'Detect'] = oo 
data2

,Date,Count,Anomaly,OutOfOrder
0,2019-09-29,11462,nan,False
1,2019-10-30,12782,nan,False
2,2019-11-28,686,anomaly,True
3,2020-02-28,13222,nan,False
4,2020-03-30,13305,nan,False
5,2020-04-29,13316,nan,False
6,2020-05-30,14016,anomaly,False
7,2020-06-29,13372,nan,True
8,2020-07-30,13487,nan,False
9,2020-08-30,13519,nan,False


In [66]:
(data.Count - data.Count.rolling(4).median()).diff(1).abs()

0         NaN
1         NaN
2         NaN
3         NaN
4       797.0
5       250.5
6       653.0
7       677.5
8        29.5
9        41.5
10       34.0
11    12867.0
12    12858.0
13       26.0
14        0.5
15        8.5
16    12908.0
17     6447.0
18    12933.0
19       12.5
Name: Count, dtype: float64